In [1]:
from mindnlp.transformers import AutoModelForSeq2SeqLM
from mindnlp.peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
from mindnlp.dataset import load_dataset

import mindspore
from mindspore import ops
import os

from mindnlp.transformers import AutoTokenizer
from mindnlp.modules.optimization import get_linear_schedule_with_warmup
from tqdm import tqdm

model_name_or_path = "t5-large"
tokenizer_name_or_path = "t5-large"
checkpoint_name = "financial_sentiment_analysis_prefix_tuning_v1.ckpt"
max_length = 128
lr = 1e-2
num_epochs = 5
batch_size = 8

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.612 seconds.
Prefix dict has been built successfully.
/home/daiyuxin/miniconda3/envs/mindspore/lib/python3.9/site-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /home/daiyuxin/miniconda3/envs/mindspore/lib/python3.9/site-packages/mindnlp/transformers/models/graphormer/algos_graphormer.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
In file included from /home/daiyuxin/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929:0,
                 from /home/daiyuxin/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/daiyuxin/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:5,
           

In [2]:
# creating model
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, num_virtual_tokens=20)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 983,040 || all params: 738,651,136 || trainable%: 0.13308583065659835


In [3]:
mindspore.dataset.config.set_seed(123)
# loading dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree")

In [4]:
classes = dataset.source.ds.features["label"].names
classes

['negative', 'neutral', 'positive']

In [5]:
train_dataset, validation_dataset = dataset.shuffle(64).split([0.9, 0.1])

[WARNING] ME(1935545:140581710448448,MainProcess):2024-05-28-16:30:53.639.132 [mindspore/dataset/engine/datasets.py:1203] Dataset is shuffled before split.


In [6]:
def add_text_label(sentence, label):
    return sentence, label, classes[label.item()]

train_dataset = train_dataset.map(add_text_label, ['sentence', 'label'], ['sentence', 'label', 'text_label'])
validation_dataset = validation_dataset.map(add_text_label, ['sentence', 'label'], ['sentence', 'label', 'text_label'])

In [7]:
next(train_dataset.create_dict_iterator())

{'sentence': Tensor(shape=[], dtype=String, value= 'The gross area of the Innova 2 project will be about 10,000 sq m ( 107,600 sq ft ) .'),
 'label': Tensor(shape=[], dtype=Int64, value= 1),
 'text_label': Tensor(shape=[], dtype=String, value= 'neutral')}

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

/home/daiyuxin/miniconda3/envs/mindspore/lib/python3.9/site-packages/mindnlp/transformers/models/t5/tokenization_t5_fast.py:162: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [9]:
import numpy as np
from mindnlp.dataset import BaseMapFunction
from threading import Lock
lock = Lock()

class MapFunc(BaseMapFunction):
    def __call__(self, sentence, label, text_label):
        lock.acquire()
        model_inputs = tokenizer(sentence, max_length=max_length, padding="max_length", truncation=True)
        labels = tokenizer(text_label, max_length=3, padding="max_length", truncation=True)
        lock.release()
        labels = labels['input_ids']
        labels = np.where(np.equal(labels, tokenizer.pad_token_id), -100, labels)
        return model_inputs['input_ids'], model_inputs['attention_mask'], labels


def get_dataset(dataset, tokenizer, shuffle=True):
    input_colums=['sentence', 'label', 'text_label']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    if shuffle:
        dataset = dataset.shuffle(64)
    dataset = dataset.batch(batch_size)
    return dataset

train_dataset = get_dataset(train_dataset, tokenizer)
eval_dataset = get_dataset(validation_dataset, tokenizer, shuffle=False)

In [10]:
next(train_dataset.create_dict_iterator())

{'input_ids': Tensor(shape=[8, 128], dtype=Int64, value=
 [[  37, 8690,  616 ...    0,    0,    0],
  [6851,   48,  993 ...    0,    0,    0],
  [  37, 3334,  349 ...    0,    0,    0],
  ...
  [4980,   15,    9 ...    0,    0,    0],
  [9273, 2699, 3751 ...    0,    0,    0],
  [  37, 8148,  349 ...    0,    0,    0]]),
 'attention_mask': Tensor(shape=[8, 128], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 'labels': Tensor(shape=[8, 3], dtype=Int64, value=
 [[7163,    1, -100],
  [7163,    1, -100],
  [7163,    1, -100],
  ...
  [7163,    1, -100],
  [1465,    1, -100],
  [7163,    1, -100]])}

In [11]:
# optimizer and lr scheduler
optimizer = mindspore.experimental.optim.AdamW(model.trainable_params(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataset) * num_epochs),
)

In [12]:
# training and evaluation
def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

grad_fn = mindspore.value_and_grad(forward_fn, None, model.trainable_params())

def train_step(**batch):
    loss, grads = grad_fn(**batch)
    optimizer(grads)
    return loss

for epoch in range(num_epochs):
    model.set_train()
    total_loss = 0
    train_total_size = train_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        loss = train_step(**batch)
        total_loss += loss.float()
        lr_scheduler.step()

    model.set_train(False)
    eval_loss = 0
    eval_preds = []
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        with mindspore._no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.float()
        eval_preds.extend(
            tokenizer.batch_decode(ops.argmax(outputs.logits, -1).asnumpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataset)
    eval_ppl = ops.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataset)
    train_ppl = ops.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|███████████████████████████████████████████████████████████████████████████████████| 29/29 [00:07<00:00,  3.89it/s]


epoch=0: train_ppl=Tensor(shape=[], dtype=Float32, value= 4.89876) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 1.58898) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.16772) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.155056)


100%|███████████████████████████████████████████████████████████████████████████████████| 29/29 [00:06<00:00,  4.17it/s]


epoch=1: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.11439) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.108311) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.09277) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0887188)


100%|███████████████████████████████████████████████████████████████████████████████████| 29/29 [00:07<00:00,  4.01it/s]


epoch=2: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.08868) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0849638) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.07994) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0769094)


100%|███████████████████████████████████████████████████████████████████████████████████| 29/29 [00:07<00:00,  4.07it/s]


epoch=3: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.07052) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0681448) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.05079) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0495383)


100%|███████████████████████████████████████████████████████████████████████████████████| 29/29 [00:06<00:00,  4.44it/s]

epoch=4: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.06582) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0637434) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.04741) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0463233)


In [13]:
# print accuracy
correct = 0
total = 0

ground_truth = []

for pred, data in zip(eval_preds, validation_dataset.create_dict_iterator(output_numpy=True)):
    true = str(data['text_label'])
    ground_truth.append(true)
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{ground_truth[:10]=}")

accuracy=93.80530973451327 % on the evaluation dataset
eval_preds[:10]=['neutral', 'neutral', 'neutral', 'neutral', 'positive positive', 'neutral', 'neutral', 'positive', 'positive', 'positive']
ground_truth[:10]=['neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'positive']


In [14]:
next(eval_dataset.create_tuple_iterator())

[Tensor(shape=[8, 128], dtype=Int64, value=
 [[   37,   804,   594 ...     0,     0,     0],
  [   37,  3829,    29 ...     0,     0,     0],
  [   37,   349,     3 ...     0,     0,     0],
  ...
  [   94,    19,  1316 ...     0,     0,     0],
  [ 1615,    59, 10006 ...     0,     0,     0],
  [   37,  1170,    13 ...     0,     0,     0]]),
 Tensor(shape=[8, 128], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[8, 3], dtype=Int64, value=
 [[7163,    1, -100],
  [7163,    1, -100],
  [7163,    1, -100],
  ...
  [7163,    1, -100],
  [7163,    1, -100],
  [1465,    1, -100]])]

In [15]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

In [16]:
ckpt = f"{peft_model_id}/adapter_model.ckpt"
!du -h $ckpt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


3.8M	t5-large_PREFIX_TUNING_SEQ_2_SEQ_LM/adapter_model.ckpt


In [12]:
from mindnlp.peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [13]:
model.set_train(False)
example = next(validation_dataset.create_dict_iterator(output_numpy=True))

print(example['text_label'])
inputs = tokenizer(example['text_label'], return_tensors="ms")
print(inputs)

with mindspore._no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.asnumpy(), skip_special_tokens=True))

neutral
{'input_ids': Tensor(shape=[1, 2], dtype=Int64, value=
[[7163,    1]]), 'attention_mask': Tensor(shape=[1, 2], dtype=Int64, value=
[[1, 1]])}


/home/daiyuxin/miniconda3/envs/mindspore/lib/python3.9/site-packages/mindnlp/transformers/generation/utils.py:1402: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://hf-mirror.com/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


[[   0 7163    1]]
['neutral']
